In [3]:
# SSNP-06.py
# 2018-08-12 Python3, 1.0
#
# read dem-all-3.csv file
# x,y,z = np.loadtxt(path_data + 'dem-all-2.csv', unpack=True) # transposed array, work for space delimeter
# https://gis.stackexchange.com/questions/66367/display-a-georeferenced-dem-surface-in-3d-matplotlib
# https://stackoverflow.com/questions/12904912/how-to-set-camera-position-for-3d-plots-using-python-matplotlib
# https://matplotlib.org/users/colormaps.html
# https://earthscience.stackexchange.com/questions/12057/how-to-interpolate-scattered-data-to-a-regular-grid-in-python
#
# run on ubuntu 16.04 LTS
# 程式檔 path is '/home/tour945/0_SSNP3D/code'
# 資料檔 path is '/home/tour945/0_SSNP3D/data'
# 圖形檔 path is '/home/tour945/0_SSNP3D/graphic'
#
% matplotlib inline
import csv
import numpy as np
import pprint
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fontManager
#
from mpl_toolkits.mplot3d.axes3d import *
from shutil import copyfile
from matplotlib import cm # colormap
# from scipy.interpolate import griddata
#
def set_graphic_area(width,height) :
        
    cm2inch = 1/2.54    # inch per cm
    #
    # define graphic area
    #
    left_margin = 2.0   # cm
    right_margin = 2.0  # cm
    #
    figure_width  = width  # cm , from xmin to xmax
    figure_height = height # cm , from ymin to ymax
    #
    top_margin = 2.0    # cm
    bottom_margin = 2.0 # cm
    #
    box_width = left_margin + figure_width + right_margin   # cm
    box_height = top_margin + figure_height + bottom_margin # cm
    #
    top_value    = 1.0 - top_margin / box_height
    bottom_value = bottom_margin / box_height
    left_value   = left_margin / box_width
    right_value  = 1.0 - right_margin / box_width
    #
    return (box_width*cm2inch,box_height*cm2inch,top_value,bottom_value,left_value,right_value,width)
#
# end of def
#
matplotlib.rcParams['pdf.fonttype'] = 42
myfont = fontManager.FontProperties(
    fname='/home/tour945/miniconda3/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/MSJH.TTC')
# -----------------------------------------------------------
#
# Left Down point: 0.0 0.0
# Right Up point:  60.0 111.8
# height (min, max) -0.0453 5.3581
#
tup7 = set_graphic_area(112,112)
#
fig = plt.figure(figsize=(tup7[0], tup7[1]))
ax  = fig.add_subplot(1,1,1, projection='3d')
fig.subplots_adjust(
                top    = tup7[2] ,
                bottom = tup7[3] ,
                left   = tup7[4] ,
                right  = tup7[5] ,
                )
#
plt.xlabel('X座標', fontproperties=myfont, fontsize=18)
plt.ylabel('Y座標', fontproperties=myfont, fontsize=18)
ax.set_zlabel('Z座標', fontproperties=myfont, fontsize=18)
#
plt.xlim(0, 40)    # set xlim to xMinMax[0], xMinMax[1]
plt.ylim(0, 112)    # set ylim to yMinMax[0], yMinMax[1]
ax.set_zlim(-1,60)
#
x_ticks = np.linspace(0,40,9)
y_ticks = np.linspace(0,112,23)
z_ticks = np.linspace(-1,60,13)
#
plt.xticks(x_ticks, fontsize=16)
plt.yticks(y_ticks, fontsize=16)
#
#
plt.grid(False)
plt.gca().set_aspect('equal', adjustable='box') # set X,Y same ratio & scale
#
path_data    = '/home/tour945/0_SSNP3D/data/'
path_graphic = '/home/tour945/0_SSNP3D/graphic/'
#
# 逐筆讀入資料 dem-all-3.csv (91584 row)
# Read from the file into the array data(:,:)
# array data shape is (91584, 3)
#
# Columns.
# data[..., 0] is array of float number
#
# 台灣橫麥卡托二度分帶投影座標系統（TM2）
#
function_float_convertTo_int = np.vectorize(np.int)
#
data2 = np.loadtxt(path_data + 'dem-all-3.csv',delimiter=',',dtype='float')
TM2_X = data2[..., 0] # 二度分帶 X座標
TM2_Y = data2[..., 1] # 二度分帶 Y座標
TM2_Z = data2[..., 2] # DEM file z value, float type
#
min_item = np.amin(data2, axis=0) # axis=0 --> min of each column
max_item = np.amax(data2, axis=0) # axis=0 --> max of each column
print ('Left Down point:', min_item[0], min_item[1])
print ('Right Up point: ' , max_item[0], max_item[1])
print ('height (min, max)', min_item[2], max_item[2])
print ('\n')
#
# ax.scatter3D(TM2_X, TM2_Y, TM2_Z, c=TM2_Z, cmap=plt.cm.jet, alpha=0.8)     # good
# ax.scatter3D(TM2_X, TM2_Y, TM2_Z, c=TM2_Z, cmap=plt.cm.viridis, alpha=0.8) 
# ax.scatter3D(TM2_X, TM2_Y, TM2_Z, c=TM2_Z, cmap=plt.cm.inferno, alpha=0.8) 
# ax.scatter3D(TM2_X, TM2_Y, TM2_Z, c=TM2_Z, cmap=plt.cm.brg, alpha=0.8) 
# ax.scatter3D(TM2_X, TM2_Y, TM2_Z, c=TM2_Z, cmap=plt.cm.gist_earth, alpha=0.8) 
# ax.scatter3D(TM2_X, TM2_Y, TM2_Z, c=TM2_Z, cmap=plt.cm.terrain, alpha=0.8) # good
# ax.view_init(elev=32, azim=-110) # view_init(elev,azim)两个参数
# ax.view_init(elev=35, azim=-110) # view_init(elev,azim)两个参数
#
# craation of a 2D grid
xi = np.linspace(min(TM2_X), max(TM2_X),200)
yi = np.linspace(min(TM2_Y), max(TM2_Y),5910)
X, Y = np.meshgrid(xi, yi)
# interpolation
Z = griddata(TM2_X, TM2_Y, TM2_Z, xi, yi)
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.jet, linewidth=1, antialiased=True)
plt.draw()
#
"""
# plt.ion()   #将画图模式改为交互模式
for angle in range(0, 360):
    ax.view_init(25, angle)
    savefig("movie%d.png" % angle)
#
"""
# 設定子圖的標題
ax.set_title('SSNP-06 台灣橫麥卡托二度分帶投影座標系統（TM2）', fontproperties=myfont, fontsize=18)
#
# bbox_inches='tight' is for display ylabel
# plt.savefig(path_graphic + "Hipparcos-02.png",format="png",dpi=72, bbox_inches='tight')
plt.savefig(path_graphic + "SSNP-06.pdf", dpi=300, bbox_inches='tight')
#
print ('Done')

ModuleNotFoundError: No module named 'scipy'